In [138]:
import sys
import os

sys.path.append(os.path.abspath("/home/miguelmd/projects/big-data-bowl-2026-analytics"))

from src.io.datasets import CSVReader
from src.core.settings import get_settings

import pandas as pd
pd.set_option('display.max_colwidth', None)

settings = get_settings()

In [139]:
players = CSVReader().read(settings.PLAYERS_FILE)
scores = CSVReader().read(settings.SCORES_FILE)
plays = CSVReader().read(settings.CLEANED_PLAYS_FILE)
inferences = CSVReader().read(settings.INFERENCE_RESULTS_FILE)

2025-12-11 17:50:42.052 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/cleaned/players.csv
2025-12-11 17:50:42.142 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/scores.csv
2025-12-11 17:50:42.164 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/cleaned/plays.csv
2025-12-11 17:50:42.235 | DEBUG    | src.io.datasets:read:23 - Reading data from /home/miguelmd/projects/big-data-bowl-2026-analytics/data/processed/inference/results.csv


In [140]:
top_fakes = scores.sort_values(by="deception_score", ascending=False)
top_fakes = top_fakes.merge(plays[["game_id", "play_id", "play_description", "pass_result"]], on=["game_id", "play_id"], how="left")

In [141]:
play = top_fakes[(top_fakes['game_id'] == 2023120308) & (top_fakes['play_id'] == 3562)]
play

,game_id,play_id,defender_id,receiver_id,deception_score,recovery_score,play_description,pass_result
1872,2023120308,3562,44878,53463,-0.228988,-0.182208,(14:15) (Shotgun) J.Flacco pass short right to E.Moore pushed ob at CLV 44 for 8 yards (A.Witherspoon).,C


In [142]:
good_fake = inferences[(inferences['game_id'] == 2023120308) & (inferences['play_id'] == 3562)]
good_fake = good_fake.reset_index(drop=True)
good_fake

,game_id,play_id,frame_id,receiver_id,defender_id,pass_result,non_completion_probability
0,2023120308,3562,1,53463,44878,C,0.367337
1,2023120308,3562,2,53463,44878,C,0.348980
2,2023120308,3562,3,53463,44878,C,0.366128
3,2023120308,3562,4,53463,44878,C,0.347517
4,2023120308,3562,5,53463,44878,C,0.404323
5,2023120308,3562,6,53463,44878,C,0.372931
6,2023120308,3562,7,53463,44878,C,0.391319
7,2023120308,3562,8,53463,44878,C,0.458247
8,2023120308,3562,9,53463,44878,C,0.386005
9,2023120308,3562,10,53463,44878,C,0.305332


In [ ]:
from matplotlib.animation import PillowWriter, FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
from src.metrics.deception import detect_change_point

change_point = detect_change_point(good_fake, "non_completion_probability")

# Define colors
background_color = "#1D384B"  # Dark Gray for background
grid_color = "#9693A4"  # Light Gray for grid
line_color = "#AB1951"  # Team Red (or any bold color)
last_point_color = "#B21952"  # White for the last point to make it stand ou

non_completion_prob = good_fake['non_completion_probability'].values
time = good_fake['frame_id'].values

fig, ax = plt.subplots(figsize=(10, 6))
fig.patch.set_facecolor(background_color)
ax.set_facecolor(background_color)

# Title and Labels
ax.set_title('Non-Completion Probability (Pre-Throw)', fontsize=18, color='white', pad=10)
# ax.set_xlabel('Frame ID', fontsize=14)
# ax.set_ylabel('Non-Completion Probability', fontsize=16)

# Axis Limits
ax.set_xlim(time.min(), time.max()+0.5)
ax.set_ylim(-0.05, 1.05)

# Remove x-axis labels (ticks)
ax.set_xticks([])

# Y-axis ticks on the right side with white color
ax.yaxis.set_ticks_position('right')  # Move y-ticks to the right
ax.set_yticks(np.linspace(0, 1, 6))  # Explicitly set y-ticks: 0.0, 0.2, 0.4, 0.6, 0.8, 1.0
ax.tick_params(axis='y', labelcolor=grid_color, labelsize=12)

# Grid with a light gray color
ax.grid(True, linestyle='--', color=grid_color, alpha=0.6)

non_completion_line, = ax.plot(
    [], [],
    lw=5,
    color=line_color, # A deep red/orange color
    label='Non-Completion Prob',
)

last_point, = ax.plot([], [], 'o', color=last_point_color, markersize=12)
change_point_line, = ax.plot([], [], color='white', lw=2, linestyle='--')  # Vertical line at change point
break_point_label = ax.text(0, -0.01, 'Break Point', fontsize=12, color='white', ha='center', va='bottom')

# Optional: Add a Legend
# ax.legend(loc='upper right', frameon=False) 

# Add extra frames for holding the last frame longer
extra_frames = 12  # Number of extra frames to hold the last frame
extended_time = np.concatenate([time, np.full(extra_frames, time[-1])])  # Extend time with repeating last value
extended_prob = np.concatenate([non_completion_prob, np.full(extra_frames, non_completion_prob[-1])])  # Repeat last value for prob

def update(frame):
    # Update the horizontal line to the current frame
    non_completion_line.set_data(extended_time[:frame], extended_prob[:frame])

    # Ensure that last_point always receives a sequence, even if it's just one point
    last_point.set_data([extended_time[frame-1]], [extended_prob[frame-1]])

    if frame >= change_point:
        change_point_line.set_data([time[change_point], time[change_point]], [0, 1])  # Vertical line at change point
        break_point_label.set_position((time[change_point], -0.05))  # Position the label slightly below the axis
        break_point_label.set_visible(True)
    else:
        change_point_line.set_data([], [])  # Hide line before reaching the change point
        break_point_label.set_visible(False)

    return non_completion_line, last_point

# fig.tight_layout()
ax.spines['top'].set_visible(False)  # Removes the top border
ax.spines['right'].set_visible(False)  # Removes the right border
ax.spines['left'].set_visible(False)  # Removes the left border
ax.spines['bottom'].set_visible(False)  # Removes the bottom border
ani = FuncAnimation(
    fig, 
    update, 
    frames=len(extended_time),  # Use extended time array to hold the last frame
    blit=True, 
    interval=100  # Default interval, can be adjusted for smoothness
)

# Save the GIF (you may want to increase fps for a smoother result)
ani.save("2023120308_3562_non_completion.gif", writer=PillowWriter(fps=10, bitrate=1800))

# Close the figure to free up memory
plt.close(fig)
print("Animation saved as '2023120308_3562_non_completion.gif'")

Animation saved as '2023120308_3562_non_completion.gif'
